### Example Naive bayes in text processing on Sastrawi and Nltk

In [ ]:
import pandas as pd
import nltk
import re
import requests
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
products = pd.read_csv('../handle_sentiment/dataset/products_terbaruu.csv')
reviews = pd.read_csv('../handle_sentiment/dataset/reviews_.csv')

In [ ]:
products = products[['id', 'name', 'price', 'rating', 'slug_product', 'image_product']]
reviews = reviews[['product', 'name','comment', 'rating', 'sentiment', 'user']]

In [ ]:
reviews['sentiment'].value_counts()

In [ ]:
plt.subplot(1, 2, 2)
reviews['sentiment'].value_counts().plot(kind='bar')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Sentiment Rating Count')

plt.tight_layout()
plt.show()

### word cloud positif

In [ ]:
# wordcloud review
data_pos = reviews[reviews['sentiment'] == 'positif']

all_text = ' '.join(word for word in data_pos['comment'])
wordcloud = WordCloud(colormap='Greens', width=1000, height=800, mode='RGBA', background_color='white').generate(all_text)

plt.figure(figsize=(20,10), dpi=80)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()





In [ ]:
data_neg = reviews[reviews['sentiment'] == 'negatif']

all_text = ' '.join(word for word in data_neg['comment'])
wordcloud = WordCloud(colormap='Reds', width=1000, height=800, mode='RGBA', background_color='white').generate(all_text)

plt.figure(figsize=(20,10), dpi=80)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()


### Teks Cleaning

In [ ]:
# cleaning text
def cleaning_text(text):
    # remove url
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text =  url_pattern.sub(r'', text)

    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)

    # remove mention handle user (@)
    text = re.sub(r'@[\w]*', ' ', text)

    # remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, " ")

    # remove extra whitespace
    text = text.strip()

    # lowercase
    text = text.lower()
    return text

### Custom Stopword List for Indonesian Text Preprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword,
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# combine stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

custom_st = '''
Bagus sekali, Murah sedang ada promo. Barang OK, sudah dipakai lancar jaya, pengiriman cepat, semoga awet. Terima kasih seler terima kasih kurir.
'''

# create dictionary with unique stopword
st_words = set(stopwords_l)
custom_stopword = set(custom_st.split())

# result stopwords
stop_words = st_words | custom_stopword
print(f'Stopwords: {list(stop_words)[:5]}')

### remove stopwords

In [ ]:
def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

### stemming and lemmatization

In [ ]:
def stemming_and_lemmatization(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)

### tokenization

In [ ]:
def tokenize(text):
    return word_tokenize(text)

### Example Text Preprocessing Pipeline for Indonesian Text Data

In [ ]:
# example
text = 'Produk oke sesuai deskripsi dan normal berfungsi, pengiriman rapih dan buble wrap, dicoba langsung oke, respond sangat cepat, maaf baru beri nilai, belum sempat coba.. Recommend.. Dan langganan ahhh, sukses seller n shoope.'
print(f'Original text: \n{text}\n')

# cleaning text and lowercase
text = cleaning_text(text)
print(f'Cleaned text: \n{text}\n')

# remove stopwords
text = remove_stopword(text)
print(f'Removed stopword: \n{text}\n')

# stemming and lemmatization
text = stemming_and_lemmatization(text)
print(f'Stemmed and lemmatized: \n{text}\n')

# tokenization
text = tokenize(text)
print(f'Tokenized: \n{text}')

### pipeline preprocess

In [ ]:
def text_preprocessing_indonesia(text):
    # cleaning text and lowercase
    output = cleaning_text(text)

    # remove stopwords
    output = remove_stopword(output)

    # stemming and lemmatization
    output = stemming_and_lemmatization(output)

    # tokenization
    output = tokenize(output)

    return output

### test pipeline preprocess

In [ ]:
text = 'Produk oke sesuai deskripsi dan normal berfungsi, pengiriman rapih dan buble wrap, dicoba langsung oke, respond sangat cepat, maaf baru beri nilai, belum sempat coba.. Recommend.. Dan langganan ahhh, sukses seller n shoope.'
text_preprocessing_indonesia(text)

### Preprocessing Text Data in a DataFrame

In [ ]:
reviews['comment'] = reviews['comment'].apply(text_preprocessing_indonesia)

### Balancing the Dataset for Sentiment Analysis

In [ ]:
negative_reviews = reviews[reviews['sentiment'] == 'negatif'].sample(n=min(620, len(reviews[reviews['sentiment'] == 'negatif'])), replace=True)
positive_reviews = reviews[reviews['sentiment'] == 'positif'].sample(n=min(620, len(reviews[reviews['sentiment'] == 'positif'])), replace=True)
sampled_reviews = pd.concat([negative_reviews, positive_reviews])

### Encoding Sentiment Labels for Machine Learning

In [ ]:
encoder = LabelEncoder()
sampled_reviews['sentiment_encoded'] = encoder.fit_transform(sampled_reviews['sentiment'])


### Splitting the Dataset for Model Training and Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sampled_reviews['comment'], sampled_reviews['sentiment_encoded'], test_size=0.2, random_state=42)

In [ ]:
X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

tfidf_vectorizer = TfidfVectorizer(max_features=1000)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

### Training a Multinomial Naive Bayes Model for Text Classification

In [ ]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

### Making Predictions with the Trained Text Classification Model

In [ ]:
y_pred = model.predict(X_test_tfidf)

### Evaluating the Model with a Confusion Matrix

In [ ]:
confusion = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

### Evaluating Sentiment Analysis Model Performance

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=encoder.classes_)

print("Sentiment Analysis Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


In [ ]:
# Menghitung metrik evaluasi
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Membuat tabel evaluasi
evaluation_table = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Score': [accuracy, precision, recall, f1]
})
evaluation_table

In [ ]:
test_reviews = ["Produk ini sangat buruk! Saya sangat puas dengan kualitasnya.",
                "Barangnya datang sangat buruk, dan pelayanannya baik.",
                "Harganya sangat terjangkau mengingat kualitas yang ditawarkan"]

predicted_sentiments = []

for review in test_reviews:
    preprocessed_review = text_preprocessing_indonesia(review)
    preprocessed_review_text = ' '.join(preprocessed_review)  # Convert the list of tokens into a single string
    test_review_tfidf = tfidf_vectorizer.transform([preprocessed_review_text])
    predicted_sentiment = model.predict(test_review_tfidf)
    decoded_sentiment = encoder.inverse_transform(predicted_sentiment)
    predicted_sentiments.append(decoded_sentiment[0])

for i in range(len(test_reviews)):
    print(f"Komentar: {test_reviews[i]}")
    print(f"Prediksi Sentimen: {predicted_sentiments[i]}\n")


### Merge the 'reviews' and 'products' dataframes based on 'product'

In [ ]:
merged_df = pd.merge(reviews, products, left_on='product', right_on='id')

### Calculate the average rating for each product based on reviews

In [ ]:

average_review_ratings = merged_df.groupby('id')['rating_x'].mean().reset_index()

### Merge the average review ratings back into the 'products' dataframe

In [ ]:
products['average_review_rating'] = average_review_ratings['rating_x']

### Calculate the final combined rating as the average of product rating and average review rating

In [ ]:
products['combined_rating'] = (products['rating'] + products['average_review_rating']) / 2


### Sort products by highest predicted rating and lowest price

In [ ]:
recommended_products = products.sort_values(by=['combined_rating', 'price'], ascending=[False, True])


### Delete Average_review_ratings

In [ ]:
recommended_products = recommended_products.drop('average_review_rating', axis=1)


### Delete Rating

In [ ]:
recommended_products = recommended_products.drop('rating', axis=1)


### Rename Rating

In [ ]:
recommended_products = recommended_products.rename(columns={'combined_rating': 'rating'})

### Print Recommended_products

In [ ]:
recommended_products[['name', 'price', 'image_product','rating']]

#### Out

In [ ]:
output_file = 'recommended_products.pkl'

# Simpan DataFrame ke dalam file pickle
with open(output_file, 'wb') as file:
    pickle.dump(recommended_products, file)

print(f'Recommended products telah disimpan dalam file pickle: {output_file}')

In [ ]:
with open(output_file, 'rb') as file:
    loaded_recommended_products = pickle.load(file)



loaded_recommended_products